In [3]:
! pip install beautifulsoup4

! pip install lxml

     |████████████████████████████████| 106 kB 6.0 MB/s eta 0:00:01
     |████████████████████████████████| 5.7 MB 3.5 MB/s eta 0:00:01


In [5]:
import bs4 as bs
import urllib.request
import re

scraped_data = urllib.request.urlopen('https://en.wikipedia.org/wiki/Citizenship_(Amendment)_Act,_2019')
article = scraped_data.read()

parsed_article = bs.BeautifulSoup(article,'lxml')

paragraphs = parsed_article.find_all('p')


article_text = ""

for p in paragraphs:
    article_text += p.text
    
print(article_text)


The Citizenship (Amendment) Act, 2019 was passed by the Parliament of India on 11 December 2019. It amended the Citizenship Act of 1955 by providing a path to Indian citizenship for illegal migrants of Hindu, Sikh, Buddhist, Jain, Parsi, and Christian religious minorities, who had fled persecution from Pakistan, Bangladesh and Afghanistan before December 2014.[3][4] Muslims from those countries were not given such eligibility.[5][6][7] The act was the first time religion had been overtly used as a criterion for citizenship under Indian law.[7][a][b][c]
The Hindu nationalist Bharatiya Janata Party (BJP), which leads the Indian government,[8][9] had promised in previous election manifestos to offer Indian citizenship to members of persecuted religious minorities who had migrated from neighbouring countries.[10][11] Under the 2019 amendment, migrants who had entered India by 31 December 2014, and had suffered "religious persecution or fear of religious persecution" in their country of or

# Preprocessing

The first preprocessing step is to remove references from the article. Wikipedia, references are enclosed in square brackets. The following script removes the square brackets and replaces the resulting multiple spaces by a single space. Take a look at the script below:

In [24]:
article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)
article_text = re.sub(r'\s+', ' ', article_text)

To clean the text and calculate weighted frequences, we will create another object.

In [25]:
formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )
formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)

In [26]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kunal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kunal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

The following script performs sentence tokenization:


In [27]:
sentence_list = nltk.sent_tokenize(article_text)

To find the frequency of occurrence of each word, we use the formatted_article_text variable. We used this variable to find the frequency of occurrence since it doesn't contain punctuation, digits, or other special characters. Take a look at the following script:

In [28]:
stopwords = nltk.corpus.stopwords.words('english')

word_frequencies = {}
for word in nltk.word_tokenize(formatted_article_text):
    if word not in stopwords:
        if word not in word_frequencies.keys():
            word_frequencies[word] = 1
        else:
            word_frequencies[word] += 1

Finally, to find the weighted frequency, we can simply divide the number of occurances of all the words by the frequency of the most occurring word, as shown below:

In [29]:
maximum_frequncy = max(word_frequencies.values())

for word in word_frequencies.keys():
    word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)

# Calculating Sentence Scores
We have now calculated the weighted frequencies for all the words. Now is the time to calculate the scores for each sentence by adding weighted frequencies of the words that occur in that particular sentence. The following script calculates sentence scores:

In [30]:
sentence_scores = {}
for sent in sentence_list:
    for word in nltk.word_tokenize(sent.lower()):
        if word in word_frequencies.keys():
            if len(sent.split(' ')) < 30:
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word]
                else:
                    sentence_scores[sent] += word_frequencies[word]

# Getting the Summary
Now we have the sentence_scores dictionary that contains sentences with their corresponding score. To summarize the article, we can take top N sentences with the highest scores. The following script retrieves top 7 sentences and prints them on the screen.

In [31]:
import heapq
summary_sentences = heapq.nlargest(7, sentence_scores, key=sentence_scores.get)

summary = ' '.join(summary_sentences)
print(summary)

The BJP government first introduced a bill to amend the citizenship law in 2016, which would have made non-Muslim migrants from Pakistan, Afghanistan, and Bangladesh eligible for Indian citizenship. The BJP government introduced a bill to amend the citizenship law in 2016, which would have made non-Muslim migrants from Pakistan, Afghanistan, and Bangladesh eligible for Indian citizenship. The Act does not include migrants from non-Muslim countries fleeing persecution to India, Rohingya Muslim refugees from Myanmar, Hindu refugees from Sri Lanka, and Buddhist refugees from Tibet, China. It stated that religious minorities such as Hindus and Sikhs are persecuted in neighbouring Muslim-majority countries, and promised to fast track a path to citizenship for non-Muslim refugees. In other parts of India, protesters said the bill discriminated against Muslims and demanded that Indian citizenship to be granted to Muslim refugees and immigrants. It added the notion of "illegal immigrants" to t